In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import pandas as pd
df = pd.read_csv('politeness.tsv', sep='\t')

In [ ]:
df.head()

In [ ]:
df['txt'].tolist()[4]

In [ ]:
df['style'].value_counts()

In [ ]:
possible_labels = df['style'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
df['label'] = df['style'].replace(label_dict)

In [ ]:
df.head()

In [ ]:
df.groupby(['style', 'label', 'split']).count()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df['split']=='train'].txt.values, 
    add_special_tokens=True, 
    return_attention_mask=True,
    max_length=128,
    truncation=True, 
    padding='max_length',
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df['split']=='val'].txt.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    max_length=128,
    truncation=True, 
    padding='max_length',
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df['split']=='test'].txt.values, 
    add_special_tokens=True, 
    return_attention_mask=True,
    max_length=128,
    truncation=True, 
    padding='max_length', 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df['split']=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df['split']=='val'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df['split']=='test'].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
len(dataset_train), len(dataset_val)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 8

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import numpy as np
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# import time
# best_val_loss = float('inf')
# early_stop_cnt = 0

# for epoch in tqdm(range(1, epochs+1)):
    
#     model.train()

#     start_time = time.time()
    
#     loss_train_total = 0

#     progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
#     for batch in progress_bar:

#         model.zero_grad()
        
#         batch = tuple(b.to(device) for b in batch)
        
#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                   'labels':         batch[2],
#                  }       

#         outputs = model(**inputs)
        
#         loss = outputs[0]
#         loss_train_total += loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#         optimizer.step()
#         scheduler.step()
        
#         #progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
#     #tqdm.write(f'\nEpoch {epoch}')
    
#     loss_train_avg = loss_train_total/len(dataloader_train)            
#     #tqdm.write(f'Training loss: {loss_train_avg}')
    
#     val_loss, predictions, true_vals = evaluate(dataloader_validation)

#     if val_loss < best_val_loss:
#         early_stop_cnt = 0

#     elif val_loss >= best_val_loss:
#         early_stop_cnt += 1

#     if(val_loss<best_val_loss):
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), f'finetuned_BERT_best.model')
    
#     val_f1 = f1_score_func(predictions, true_vals)
#     #tqdm.write(f'Validation loss: {val_loss}')
#     #tqdm.write(f'F1 Score (Weighted): {val_f1}')

#     end_time = time.time()
#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
#     print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s', flush=True)
#     print(f'\tTrain Loss: {loss_train_avg:.3f}', flush=True)
#     print(f'\t Val. Loss: {val_loss:.3f}', flush=True)

#     if early_stop_cnt == 3:
#         print('Early Stoping...', flush=True)
#         break

In [ ]:
# print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
# print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
# print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

In [ ]:
# print(torch.cuda.memory_summary())

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('finetuned_BERT_best.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_test)

In [ ]:
accuracy_per_class(predictions, true_vals)

In [ ]:
lbls = [0,1,2,3,4,5,6,7,8,9]
# classes = ['P_0','P_1','P_2', 'P_3', 'P_4', 'P_5', 'P_6','P_7','P_8','P_9']
classes = ['P_2','P_7','P_0', 'P_9', 'P_5', 'P_1', 'P_8','P_6','P_4','P_3']

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

confusion = confusion_matrix(labels_flat, preds_flat, labels=lbls)
print('Confusion Matrix')
#print(confusion)

#confusion_classes_tuples = list(zip(classes, confusion))
confusion_df = pd.DataFrame(confusion,
                     index = classes,
                     columns = classes)
print(confusion_df)
#confusion_df.to_csv('confusion_matrix', sep='\t', encoding='utf-8')

print('\nClassification Report\n')
print(classification_report(labels_flat, preds_flat, target_names=classes))

In [ ]:
class_labels_dict = dict(zip(lbls,classes))

In [ ]:
def pred_label(inp):
    inputs = tokenizer(inp, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    # print(predicted_class_id)
    # print(class_labels_dict[predicted_class_id])
    # print(int(class_labels_dict[predicted_class_id].split('_')[1]))
    return int(class_labels_dict[predicted_class_id].split('_')[1])

In [ ]:
pred_label("good bye. Thank you.")

In [ ]:
import pickle

In [ ]:
def avg_polite_score(list_pkl):
    pred_direct_lists = None
    with open(list_pkl, 'rb') as f:
        pred_direct_lists = pickle.load(f)

    pred_labels = []
    for pred_direct in pred_direct_lists[:1000]:
        pred_labels.append(pred_label(pred_direct))
    print(sum(pred_labels)/len(pred_labels))

In [ ]:
avg_polite_score('../politeness_transfer/direct/src_direct_lists.pkl')

In [ ]:
avg_polite_score('../politeness_transfer/direct/trg_direct_lists.pkl')

In [ ]:
avg_polite_score('../politeness_transfer/direct/pred_direct_lists.pkl')

In [ ]:
avg_polite_score('../politeness_transfer/gen/pred_gen_lists.pkl')

In [ ]:
P_0_test = df[(df['split']=='test') & (df['style']=='P_0')].txt.values
print(len(P_0_test))

In [ ]:
avg_polite_score('P_0_test.pkl')

In [ ]:
avg_polite_score('../politeness_transfer/direct/P_0_test_polite_direct.pkl')

In [ ]:
avg_polite_score('../politeness_transfer/gen/P_0_test_polite_gen.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/responses/test_res.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/without-finetune/dlg_responses_wo_pred_lists.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/responses/test_polite_res_direct.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/responses/test_polite_res_gen.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/dlg_responses_direct_pred_lists.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune-polite/polite_dlg_responses_direct_pred_lists.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/without-finetune/dlg_responses_wo_pred_lists_dialoGPT.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/dlg_responses_direct_pred_lists_GPT2.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/without-finetune/dlg_responses_wo_pred_lists_GPT2.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/dlg_responses_direct_pred_lists_DialoGPT.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/DialoGPT_finetune_lastmodel.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/DialoGPT_finetune_lastmodel_polite.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/GPT2_finetune_last.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/GPT2_finetune_best.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/GPT2_finetune_polite_best.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/GPT2_finetune_last_polite.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/DialoGPT_finetune_best.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/DialoGPT_finetune_polite_best.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune/GPT2_finetune_tag-gen_polite.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/finetune-polite/Blenderbot_polite_pred_tag-gen.pkl')

In [ ]:
# with open(r"../../../tagger-generator/tag-and-generate-train/experiments/syn_output", 'r') as fp:
#     lines = fp.readlines()
#     with open('../politeness_transfer/synthetic_test_polite_tagandgen.pkl', 'wb') as f:
#         pickle.dump(lines,f)
# with open(r"../../../tagger-generator/tag-and-generate-train/experiments/train_output", 'r') as fp:
#     lines = fp.readlines()
#     print(len(lines))
#     with open('../dialog/daily_dialog/responses/tag-gen/train_polite_res.pkl', 'wb') as f:
#         pickle.dump(lines,f)
# with open(r"../../../tagger-generator/tag-and-generate-train/experiments/dev_output", 'r') as fp:
#     lines = fp.readlines()
#     print(len(lines))
#     with open('../dialog/daily_dialog/responses/tag-gen/dev_polite_res.pkl', 'wb') as f:
#         pickle.dump(lines,f)
# with open(r"../../../tagger-generator/tag-and-generate-train/experiments/test_output", 'r') as fp:
#     lines = fp.readlines()
#     print(len(lines))
#     with open('../dialog/daily_dialog/responses/tag-gen/test_polite_res.pkl', 'wb') as f:
#         pickle.dump(lines,f)

In [ ]:
avg_polite_score('../politeness_transfer/synthetic_test_polite_tagandgen.pkl')

In [ ]:
avg_polite_score('../dialog/daily_dialog/responses/tag-gen/test_polite_res.pkl')